In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:

from projimmo.data import hello
hello()

hello toi


In [2]:
import sys
sys.path

['/home/cpoulard/code/CecilePoulard/04-Decision-Science/01-Project-Setup/data-context-and-setup',
 '/home/cpoulard/code/CecilePoulard/projimmo',
 '/home/cpoulard/.pyenv/versions/3.10.6/lib/python310.zip',
 '/home/cpoulard/.pyenv/versions/3.10.6/lib/python3.10',
 '/home/cpoulard/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload',
 '',
 '/home/cpoulard/.pyenv/versions/3.10.6/envs/projimmo/lib/python3.10/site-packages']

In [3]:
import projimmo

projimmo.__file__

'/home/cpoulard/code/CecilePoulard/projimmo/projimmo/__init__.py'

In [2]:
import os
import pandas as pd
from projimmo.params import *
#data=os.getenv('DATA_DIR')
print(DATA_DIR)

chunk = pd.read_csv(DATA_DIR, sep="|", nrows=CHUNK_SIZE, dtype=str)
chunk.head()
#'/home/cpoulard/projimmo/raw_data/valeursfoncieres-2023.txt'


raw_data/valeursfoncieres-2023.txt


,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,1,3,Dépendance,NaN,0,0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,1,3,Dépendance,NaN,0,0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,1,2,Appartement,NaN,233,8,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,03/01/2023,Vente,...,NaN,0,1,Maison,NaN,64,3,S,NaN,988
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,0,1,Maison,NaN,73,3,S,NaN,835


In [1]:
#Ecrire dans la bdd avec BigQuery

import numpy as np
import pandas as pd

from google.cloud import bigquery
from pathlib import Path
from colorama import Fore, Style


from projimmo.params import *

TABLE='test'
table=f"{GCP_PROJECT}.{BQ_DATASET}.{TABLE}"
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})

client = bigquery.Client()

write_mode = "WRITE_TRUNCATE" # or "WRITE_APPEND"
job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

job = client.load_table_from_dataframe(df, table, job_config=job_config)
result = job.result()

In [1]:

import pandas as pd
import random
import seaborn as sns
from scipy import stats
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler,OrdinalEncoder
import pandas as pd
from pathlib import Path
from projimmo.data import *
from projimmo.preprocessor import *

query = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.DVF_{DATA_YEAR}
    """

    # $CHA_BEGIN
    # Retrieve data using `get_data_with_cache`
#data_query_cache_path = Path(LOCAL_DATA_PATH).joinpath("raw", f"query_{min_date}_{max_date}_{DATA_SIZE}.csv")
 #   data_query = get_data_with_cache(
 #       query=query,
 #       gcp_project=GCP_PROJECT,
 #       cache_path=data_query_cache_path,
 #       data_has_header=True
 #   ).joinpath("raw", f"query_{min_date}_{max_date}_{DATA_SIZE}.csv")
data_query_cache_path=Path(f"raw_data/valeursfoncieres-{DATA_YEAR}.txt")
# Lire le fichier entier
#df = pd.read_csv("raw_data/valeursfoncieres-2023.txt", sep="|", dtype=str)
print(data_query_cache_path)
df= get_data_with_cache(query=query,
gcp_project=GCP_PROJECT,
        cache_path=data_query_cache_path,
        data_has_header=True
   )


#load_data_to_bq(
#        data_processed_with_timestamp,
 #       gcp_project=GCP_PROJECT,
 #       bq_dataset=BQ_DATASET,
 #       table=f'processed_{DATA_SIZE}',
 #       truncate=True
  #  )

df_0=clean_data(df)

df.dtypes
df_1=clean_outliers(df_0)
#load_data_to_bq(
#        df,
 #       gcp_project=GCP_PROJECT,
#        bq_dataset=BQ_DATASET,
 #       table=f'clean_data_{DATA_YEAR}',
 #       truncate=True
 #   )

#df_2=preprocess_features(df_1)


raw_data/valeursfoncieres-2023.txt

Load data from local CSV...
✅ Data loaded, with shape (3727005, 43)


In [2]:
df_0.head(20)

,valeur_fonciere,type_de_voie,nombre_de_lots,code_type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,month_mutation,year_mutation,somme_surface_carrez,departement
654,230000.0,RUE,1,2,95.0,4.0,0.0,1,2023,97.70,13
1027,245000.0,RUE,2,2,95.0,4.0,0.0,1,2023,94.79,13
1598,87500.0,RUE,1,2,45.0,2.0,0.0,1,2023,46.87,13
2101,135000.0,nan,1,2,58.0,3.0,0.0,1,2023,60.30,13
2112,140000.0,RUE,1,2,65.0,3.0,0.0,2,2023,64.20,13
2268,155000.0,RUE,1,2,93.0,3.0,0.0,1,2023,109.20,13
2564,80000.0,RUE,1,2,61.0,2.0,0.0,1,2023,67.23,13
2781,230000.0,RTE,1,2,100.0,5.0,0.0,1,2023,120.67,13
2998,90000.0,RUE,1,2,80.0,4.0,0.0,1,2023,76.65,13
3374,112000.0,RTE,1,2,48.0,3.0,0.0,1,2023,39.03,13


In [2]:
df_0.dtypes

valeur_fonciere              float64
type_de_voie                  object
departement                    int64
nombre_de_lots                 int64
code_type_local                int64
surface_reelle_bati          float64
nombre_pieces_principales    float64
surface_terrain              float64
month_mutation                 int64
year_mutation                  int64
somme_surface_carrez         float64
dtype: object